In [ ]:
import numpy as np
import pygrib
from scipy import interpolate, optimize 

import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap, cm

In [ ]:
def linear_splines_unif(data, num_knots=10, level_width=1):    
    levels = np.array(range(1,100))
    if data[-1] == 0:
        return np.zeros(int(np.floor(100/level_width)))
    else:
        knot_ = np.where(data > 0)[0].min() - 1   
        if knot_ > 1:
            knots = np.unique(np.linspace(knot_-1, 98, num_knots-1, dtype=int))
            knots = np.insert(knots, 0, 0)
        else:
            knots = np.unique(np.linspace(0, 98, num_knots, dtype=int))
    # approx = interpolate.interp1d(knots+1, data[knots], assume_sorted=True) 
    # return approx(levels[level_width-1::level_width])
    return np.interp(levels[level_width-1::level_width], knots+1, data[knots])

def linear_splines(x, num_knots, *params):
    knot_vals = list(params[0][0:num_knots])
    knots = list(params[0][num_knots:])
    knots = np.insert(knots, 0, 1)
    knots = np.append(knots, 99)
    return np.interp(x, knots, knot_vals)

def linear_splines_var(data, num_knots, level_width):
    if data[49] == 0:
        count[1] += 1
        return linear_splines_unif(data, num_knots=num_knots, level_width=level_width)
    p_0 = np.linspace(1,99,num_knots).astype(int)[1:-1]
    p_0 = np.hstack([data[0], data[p_0], data[-1], p_0])
    # bounds = [1,99]*(2*num_knots - 2)
    try:
        fit, _ = optimize.curve_fit(lambda x, *params : linear_splines(x, num_knots, params),
                                    np.linspace(1,99,99), data, p_0)
        count[2] += 1
        fit = np.insert(fit, num_knots, 1)
        fit = np.append(fit, 99)
        levels = np.linspace(1,99,99)
        levels = levels[level_width-1::level_width]
        return np.interp(levels, fit[num_knots:], fit[:num_knots])
    except RuntimeError:
        count[0] += 1
        return linear_splines_unif(data, num_knots=num_knots, level_width=level_width)

In [ ]:
fn_grb = 'blend.t00z.qmd.f012.co.grib2'
ds_grb = pygrib.open(fn_grb)
lat, long = ds_grb.message(2).data()[1:]
precip_shape = lat.shape
precip_levels = np.zeros(shape=(99,)+precip_shape)
for i in range(99):
    precip_levels[i,:,:] = ds_grb.message(i+2).data()[0]

In [ ]:
# level_width = 30
# precip_levels_approx = np.zeros(shape=(int(np.floor(100/level_width)),)+lat.shape)
# for i in range(precip_shape[0]):
#     for j in range(precip_shape[1]):
#         precip_levels_approx[:,i,j] = linear_splines_unif(data=precip_levels[:,i,j], 
#                                     num_knots=10, return_vals=True, level_width=level_width)      
# np.save('precip_levels_approx', precip_levels_approx)
precip_levels_approx = np.load('precip_levels_approx.npy')

In [ ]:
count = [0,0,0]
level_width = 30
precip_levels_approx_var = np.zeros(shape=(int(np.floor(100/level_width)),)+lat.shape)
for i in range(precip_shape[0]):
    for j in range(precip_shape[1]):
        precip_levels_approx_var[:,i,j] = linear_splines_var(precip_levels[:,i,j], 10, level_width)
np.save('precip_levels_approx_var', precip_levels_approx_var)
# precip_levels_approx_var = np.load('precip_levels_approx_var.npy')

In [ ]:
# graphing using basemap

def Basemap_plot(data, long, lat, var_name='Precipitation', diff=False, levels=False):

    map = Basemap(llcrnrlon=-123.,llcrnrlat=20., 
                   urcrnrlon=-59., urcrnrlat=48., 
                   projection='lcc', 
                   lat_1=38.5,
                   lat_0=38.5,
                   lon_0=-97.5,
                   resolution='l')

    # draw coastlines, country boundaries, fill continents
    map.drawcoastlines(linewidth=0.25)
    map.drawcountries(linewidth=0.25)
    map.fillcontinents(color='xkcd:white',lake_color='xkcd:blue')



    # draw the edge of the map projection region (the projection limb)
    map.drawmapboundary(fill_color='xkcd:blue')
    map.drawstates()

    # draw lat/lon grid lines every 30 degrees.
    map.drawmeridians(np.arange(-180,180,30))
    map.drawparallels(np.arange(-90,90,30))

    x, y = map(long, lat)
    
    if diff:
        data_abs_max = int(np.ceil(max(np.abs(data.min()),np.abs(data.max()))))
        if not(isinstance(levels, list) or isinstance(levels, np.ndarray)):
            levels = list(range(-data_abs_max,data_abs_max+1))
        plt.pcolormesh(x, y, data,
                       norm=colors.Normalize(vmin=-data_abs_max, vmax=data_abs_max),
                       cmap='seismic', shading='nearest')
        # map.contourf(x, y, data, 16, levels=levels, cmap='seismic')
        map.colorbar()
    else:
        map.contour(x, y, data, 16, linewidths=1.5)
    plt.title(var_name)
    plt.show()

In [ ]:
var_name = 'Precipitation at Probability Level 30%'
Basemap_plot(data=precip_levels[30-1,:,:], long=long, lat=lat, var_name=var_name)

In [ ]:
var_name = 'Difference at Probability Level 90%'
diff = precip_levels_approx[2,:,:]-precip_levels[90-1,:,:]
Basemap_plot(data=diff, long=long, lat=lat, var_name=var_name, diff=True)

In [ ]:
loc = [int(np.random.uniform(0,lat.shape[0]-1)),int(np.random.uniform(0,lat.shape[1]-1))]
plt.figure()
plt.plot(np.linspace(1,99,99),precip_levels[:,loc[0],loc[1]])
plt.title(f'CDF a latitude {lat[loc[0],loc[1]]} and longitude {long[loc[0],loc[1]]}')
plt.xlabel('probability levels')
plt.ylabel('mm of precipitation')
plt.show()